In [1]:
import psycopg

In [7]:
def start_planning(year, quarter, user, pwd):
    arg = f'dbname=adventureworkslt user={user} password={pwd} host=localhost'
    quarter_id = '.'.join(list(map(str, [year, quarter])))
    con = psycopg.connect(arg)
    ### droping data from "plan_data"
    with con.cursor() as cur:
        ### droping data from "plan_data"
        cur.execute(f'''delete from plan_data
                        where quarterid = cast({quarter_id} as text)''')
        ### droping data from "plan_status"
        cur.execute(f'''delete from plan_status
                        where quarterid = cast({quarter_id} as text)''')
        ### inserting data into "plan_data"
        cur.execute(f'''insert into plan_data (versionid, country, quarterid, pcid, salesamt) select 'N' as versionid, countrycode, {year}.{quarter}, categoryid as pcid, salesamt from (WITH subquery1 AS (select cid, salesamt as sales1, quarter_yr, categoryid from company_sales
where year={year-1} and ccls in ('A', 'B') and quarter_yr={quarter}),
subquery2 AS (select cid, salesamt as sales2, quarter_yr, categoryid from company_sales
where year={year-2} and ccls in ('A', 'B') and quarter_yr={quarter})
SELECT
    COALESCE(sq1.cid, sq2.cid) AS cid,
    COALESCE(sq1.quarter_yr, sq2.quarter_yr) AS quarterid,
    COALESCE(sq1.categoryid, sq2.categoryid) AS categoryid,
    case when sq1.sales1 = 0 and sq2.sales2 = 0 then 0
        else (COALESCE(sq1.sales1, 0) + COALESCE(sq2.sales2, 0)) / 2 end as salesamt
FROM subquery1 sq1
FULL outer JOIN subquery2 sq2
    ON sq1.cid = sq2.cid
    AND sq1.quarter_yr = sq2.quarter_yr
    AND sq1.categoryid = sq2.categoryid) as sq3
join company c on c.id=sq3.cid''')
        ### create copy P
        con.execute()
        ### filling plan_status
        con.execute('''insert into plan_status ()''')





start_planning(2023, 1, 'postgres', 'djigurdgen43')